In [1]:
import numpy as np
import plotly.graph_objects as go
import requests
import json
import datetime
import csv
import codecs
import urllib
import matplotlib.pyplot as plt
import pandas as pd
import time
import glob

In [2]:
def load_data(fname):
    data = pd.read_csv(fname, skiprows=1)
    f = open(fname, 'rb')
    line = f.readline()
    f.close()
    line = line[1:].decode().strip()
    times = line.split(',')
    times = [int(times[0]), int(times[1])]
    # cleanup whitespaces in column names
    data.rename(columns=lambda x: x[1:].strip() if x.startswith("#") else x.strip(), inplace=True)
    # cleanup whitespaces in data
    data = data.applymap(lambda x: x.strip() if type(x)==str else x)
    #. Add columns in which the time on the dongle is converted UNIX EPOCH time, and "datetime64"
    data['time'] = (data['dongle_time']-times[0])/1000 + times[1] - 6*3600
    data['datetime'] = data['time'].astype('datetime64[s]')
    return data

In [3]:
data = load_data('A.csv')

In [7]:
def plotly_rssi_ch(t,rssi, ch, f):
    #. Adapted from https://github.com/plotly/plotly.py/issues/1445
    fig = go.FigureWidget()
    ch_list = [37, 38, 39]
    colors = ['red', 'blue', 'green']
    count = 0
    for c, color in zip(ch_list, colors):
        b = ch==c
    
        fig.add_scattergl(x=t[b], y=rssi[b]+0.25*(c-38), 
                          mode='markers', name='ch %d'%c, 
                          marker = dict(size=5, symbol='cross', color=color)
                         )
        fig.add_histogram(y=rssi[b], name='rssi', marker=dict(color=color, opacity=0.7), 
                          xaxis='x2'
                         )
        fig.data[2*count+1].meta = rssi[b]
        count += 1

#     b = ch==37
#     fig.add_scattergl(x=t[b], y=rssi[b]+0.25, mode='markers', name='rssi_vs_time', 
#                       marker = dict(size=5, symbol='cross', color='red')
#                      )    
#     b = ch==39
#     fig.add_scattergl(x=t[b], y=rssi[b]-0.25, mode='markers', name='rssi_vs_time', 
#                       marker = dict(size=5, symbol='cross', color='green')
#                      )    
    #. Use 'meta' field to hold raw data
    fig.layout = dict(xaxis=dict(domain=[0, 0.84], showgrid=False, zeroline=False),
                      yaxis=dict(domain=[0, 1], showgrid=False, zeroline=False),
                      showlegend=True,
                      margin=dict(t=50),
                      hovermode='closest',
                      bargap=0.25,
                      xaxis2=dict(domain=[0.85, 1], showgrid=False, zeroline=False),
                      height=600,
                      title={'text':f, 'x':0.42, 'xanchor':'center'}
                     )
    #  Lots of trouble getting the updating the histogram to work... Seems to work now that
    #.  I have wrapped the whole thing in a function... Works on local jupyter
    #.  Not working colab jupyter
    # 
    def do_zoom(layout, xaxis_range, yaxis_range):
        #global y
        inds = ((xaxis_range[0] <= t) & (t <= xaxis_range[1]) &
                (yaxis_range[0] <= rssi) & (rssi <= yaxis_range[1]))

        with fig.batch_update():
            fig.data[1].y = fig.data[1].meta[inds[ch==37]]
            fig.data[3].y = fig.data[3].meta[inds[ch==38]]
            fig.data[5].y = fig.data[5].meta[inds[ch==39]]


    fig.layout.on_change(do_zoom, 'xaxis.range', 'yaxis.range')

    return fig


In [ ]:
data_by_mac = {}
for m in np.unique(data['mac']):
    data_by_mac[m] = data[data['mac']==m]
    fig = plotly_rssi_ch(data_by_mac[m]['datetime'], 
                        data_by_mac[m]['rssi'], 
                        data_by_mac[m]['ch'],
                        m)
    fig.show()

In [ ]:
#matplotlib plotting
#
def plot_rssi(t, rssi):
    # compute histogram fast
    hist = np.array(np.unique(rssi, return_counts=True)).T
    
    # definitions for the axes
    left, width = 0.1, 0.65
    bottom, height = 0.1, 0.65
    spacing = 0.005


    rect_scatter = [left, bottom, width, height]
    rect_histy = [left + width + spacing, bottom, 0.2, height]

    # start with a rectangular Figure
    plt.figure(figsize=(16, 8))

    ax_scatter = plt.axes(rect_scatter)
    ax_scatter.tick_params(direction='in', top=True, right=True)

    ax_histy = plt.axes(rect_histy)
    ax_histy.tick_params(direction='in', labelleft=False)

    ax_scatter.plot(t, rssi,'.',  markersize=1)

    ax_histy.barh(hist[:,0], hist[:,1])
    ax_histy.set_ylim(ax_scatter.get_ylim())
    return ax_scatter, ax_histy
    #plt.show()